<a href="https://colab.research.google.com/github/Gizmo-Labs/Statistical-Computing/blob/master/IdealerWohnort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Statistical Computing - Capstone Project

## Explorative Data Analysis

## ▶️ In the first step we clone our Github-Repository to get the project structure

In [2]:
!git clone https://github.com/Gizmo-Labs/Statistical-Computing.git

Cloning into 'Statistical-Computing'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 24 (delta 4), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (24/24), 476.39 KiB | 1.45 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [4]:
import pandas as pd


## ▶️ Second step is to import each .csv-file in a single pandas dataframe

In [87]:
# Import the .csv-file with the weather datasets
time_series_df = pd.read_csv('/content/Statistical-Computing/data/data.csv', index_col=False)

# Dropping the columns we don't need for our analysis
time_series_df = time_series_df.drop(['Qualitaet_Byte', 'Qualitaet_Niveau'], axis=1)

# Now the .csv-file with the product codes
product_code_df = pd.read_csv('/content/Statistical-Computing/data/prd.csv', index_col=False)

# Dropping the columns we don't need for our analysis
product_code_df = product_code_df.drop(['Beschreibung_DWD', 'Beschreibung_Inspire'], axis=1)

# And at last the .csv-file with the station data
station_names_df = pd.read_csv('/content/Statistical-Computing/data/sdo.csv', index_col=False)

# Dropping the columns we don't need for our analysis
station_names_df = station_names_df.drop(['Geogr_Laenge', 'Geogr_Breite', 'Hoehe_ueber_NN', 'Metadata_Link'], axis=1)


✔️ Our weather data table looks like this for the moment!

In [89]:
time_series_df.head(2)

,Produkt_Code,SDO_ID,Zeitstempel,Wert
0,OBS_DEU_P1D_SD,1228,2014-01-01,0.0
1,OBS_DEU_P1D_SD,1228,2014-01-02,0.3


✔️ Our product-code table like this!

In [90]:
product_code_df.head(2)

,Produkt_Code,Produkt_Titel,Einheit
0,OBS_DEU_P1D_SD,Daily station observations of sunshine duratio...,hours
1,OBS_DEU_P1D_T2M_X,Daily station observations of maximal temperat...,°C


✔️ And finally the station table like this!

In [91]:
station_names_df.head(2)

,SDO_ID,SDO_Name
0,1379,Flensburg (Schäferhaus)
1,7367,Alfeld


## ▶️ Third step converts our Data-Frames into a single one to simplify our further processing

In [126]:
# We combine our weather data with our product-code data in one single table
complete_values_df = pd.merge(time_series_df, product_code_df, on='Produkt_Code')

# Then we take this artifact on combine it with the station data
complete_values_df = pd.merge(complete_values_df, station_names_df, on='SDO_ID')

# The result gives us a single table where we now can see the locations and type of time-series
complete_values_df.head(2)

,Produkt_Code,SDO_ID,Zeitstempel,Wert,Produkt_Titel,Einheit,SDO_Name
0,OBS_DEU_P1D_SD,1228,2014-01-01,0.0,Daily station observations of sunshine duratio...,hours,UFS TW Ems
1,OBS_DEU_P1D_SD,1228,2014-01-02,0.3,Daily station observations of sunshine duratio...,hours,UFS TW Ems


## ▶️ Now that we have ONE single Data-Frame with our values, we have to clean it up a bit!

In [127]:
# After merging we do no longer need some columns
complete_values_df = complete_values_df.drop(['Produkt_Code', 'SDO_ID'], axis=1)

complete_values_df.head()

,Zeitstempel,Wert,Produkt_Titel,Einheit,SDO_Name
0,2014-01-01,0.0,Daily station observations of sunshine duratio...,hours,UFS TW Ems
1,2014-01-02,0.3,Daily station observations of sunshine duratio...,hours,UFS TW Ems
2,2014-01-03,0.9,Daily station observations of sunshine duratio...,hours,UFS TW Ems
3,2014-01-04,1.5,Daily station observations of sunshine duratio...,hours,UFS TW Ems
4,2014-01-05,3.1,Daily station observations of sunshine duratio...,hours,UFS TW Ems
